# NWS Prep
This notebook provides download links for NWS data then ingests the data by station/year from html files.

It handles cropping extra hours from the year (due to time zones) and some data clean-up.

Finally, it outputs a pkl ready to be analyzed and used.

---
**Imports**

In [1]:
import pandas as pd

**Function to get appropriate file names and url's**

In [2]:
def getPaths(site, startYr):
    endYr = startYr + 1
    hours = '72'
    units = 'english'
    chart = 'off'
    headers = 'none'
    obs = 'tabular'
    hourly = 'true'
    pview = 'full'
    history = 'yes'
    start = str(startYr) + '0101'
    end = str(endYr) + '0101'

    url = f'https://www.weather.gov/wrh/timeseries?site={site}&hours={hours}&units={units}&chart={chart}&headers={headers}&obs={obs}&hourly={hourly}&pview={pview}&history={history}&start={start}&end={end}'

    file_path = f'{site}-{startYr}.html'

    return {'file_path':file_path,'url':url}

## Get Each Site Wx Data

This lists every link we must load, and what to save it as.

Would love a programmatic way, but need async http calls...

In [10]:
# Earliest dates to pull:
# JVEMT: 2019
# S11MT: 2019
# SNSLP: 2007, but no 2013-2014!
# SH7MT: 2019
# SH4MT: 2019
# MRPMT: 2019

sites_years = {'JVEMT': 2019, 'S11MT': 2019, 'SNSLP': 2007, 'SH7MT': 2019, 'SH4MT': 2019, 'MRPMT': 2019}
files = set()

for site, year in sites_years.items():    
    for start in range(year, 2023):
        paths = getPaths(site, start)
        files.add(paths['file_path'])
        print(paths['file_path'],'=',paths['url'],'\n')

JVEMT-2019.html = https://www.weather.gov/wrh/timeseries?site=JVEMT&hours=72&units=english&chart=off&headers=none&obs=tabular&hourly=true&pview=full&history=yes&start=20190101&end=20200101 

JVEMT-2020.html = https://www.weather.gov/wrh/timeseries?site=JVEMT&hours=72&units=english&chart=off&headers=none&obs=tabular&hourly=true&pview=full&history=yes&start=20200101&end=20210101 

JVEMT-2021.html = https://www.weather.gov/wrh/timeseries?site=JVEMT&hours=72&units=english&chart=off&headers=none&obs=tabular&hourly=true&pview=full&history=yes&start=20210101&end=20220101 

JVEMT-2022.html = https://www.weather.gov/wrh/timeseries?site=JVEMT&hours=72&units=english&chart=off&headers=none&obs=tabular&hourly=true&pview=full&history=yes&start=20220101&end=20230101 

S11MT-2019.html = https://www.weather.gov/wrh/timeseries?site=S11MT&hours=72&units=english&chart=off&headers=none&obs=tabular&hourly=true&pview=full&history=yes&start=20190101&end=20200101 

S11MT-2020.html = https://www.weather.gov/wrh

In [11]:
# SNSLP 2013 has no data, must remove

files.discard('SNSLP-2013.html')

**Create dictionary of sites (keys) with list (value) as tuple of file name and dataframe per year**

In [18]:
# CAUTION: this cell takes 3 minutes to run!

sites_wx = {'JVEMT': [], 'S11MT': [], 'SNSLP': [], 'SH7MT': [], 'SH4MT': [], 'MRPMT': []}

for file in files:
    print(f'Getting {file}')
    df = pd.read_html('../raw_data/nws/' + file)[0] # b/c read_html gives a list of df's, and we have just one
    df['site'] = file[:5] # parse site abbreviation from first 5 chars
    df['file_name'] = file
    sites_wx[file[:5]].append((file, df))

Getting JVEMT-2022.html
Getting S11MT-2022.html
Getting SH7MT-2020.html
Getting SNSLP-2016.html
Getting JVEMT-2019.html
Getting JVEMT-2020.html
Getting S11MT-2019.html
Getting SNSLP-2018.html
Getting SNSLP-2017.html
Getting SNSLP-2014.html
Getting SNSLP-2010.html
Getting SNSLP-2020.html
Getting SNSLP-2012.html
Getting MRPMT-2020.html
Getting MRPMT-2021.html
Getting SNSLP-2011.html
Getting SNSLP-2015.html
Getting MRPMT-2019.html
Getting S11MT-2021.html
Getting SH4MT-2021.html
Getting SH4MT-2020.html
Getting SNSLP-2019.html
Getting S11MT-2020.html
Getting SNSLP-2022.html
Getting MRPMT-2022.html
Getting SNSLP-2008.html
Getting SH4MT-2022.html
Getting SNSLP-2007.html
Getting SH7MT-2021.html
Getting SNSLP-2021.html
Getting JVEMT-2021.html
Getting SH7MT-2019.html
Getting SH4MT-2019.html
Getting SNSLP-2009.html
Getting SH7MT-2022.html


In [19]:
dfs = []

for site in sites_wx:
  for yrs in sites_wx[site]:
    df = yrs[1].copy().iloc[17:-7] # trim hours due to GMT
    year = yrs[0][6:10]
    df['year'] = year # parse from file name
    dfs.append(df)

all_wx = pd.concat(dfs)
all_wx

,Date/Time (L),Temp. (°F),DewPoint(°F),RelativeHumidity(%),HeatIndex(°F),WindChill(°F),WindDirection,WindSpeed(mph),1 HourPrecip(in),SnowDepth(in),...,Snowfall24 Hour(in),site,file_name,year,Sea LevelPressure(mb),StationPressure(in Hg),AltimeterSetting(in Hg),SolarRadiation(W/m²),PercentPossible(%),Unnamed: 1
17,"Dec 23, 4:00 am",-25.0,-31.0,72.0,NaN,NaN,ESE,1G4,0.0,12.6,...,0.3,JVEMT,JVEMT-2022.html,2022,NaN,NaN,NaN,NaN,NaN,NaN
18,"Dec 23, 3:00 am",-26.0,-32.0,72.0,NaN,NaN,ESE,1G3,0.0,12.6,...,0.4,JVEMT,JVEMT-2022.html,2022,NaN,NaN,NaN,NaN,NaN,NaN
19,"Dec 23, 2:00 am",-27.0,-33.0,71.0,NaN,NaN,ESE,1G4,0.0,12.5,...,0.3,JVEMT,JVEMT-2022.html,2022,NaN,NaN,NaN,NaN,NaN,NaN
20,"Dec 23, 1:00 am",-28.0,-34.0,72.0,NaN,NaN,ESE,2G5,0.0,12.5,...,0.2,JVEMT,JVEMT-2022.html,2022,NaN,NaN,NaN,NaN,NaN,NaN
21,"Dec 23, 12:00 am",-29.0,-35.0,71.0,NaN,NaN,ESE,2G4,0.0,12.5,...,0.2,JVEMT,JVEMT-2022.html,2022,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8332,"Jan 1, 4:00 am",-4.0,NaN,NaN,NaN,-26.0,S,18G31,NaN,40.8,...,NaN,MRPMT,MRPMT-2022.html,2022,NaN,24.60,29.82,NaN,NaN,NaN
8333,"Jan 1, 3:00 am",-5.0,NaN,NaN,NaN,-26.0,S,15G21,NaN,41.5,...,NaN,MRPMT,MRPMT-2022.html,2022,NaN,24.59,29.81,NaN,NaN,NaN
8334,"Jan 1, 2:00 am",-3.0,NaN,NaN,NaN,-23.0,SSW,15G25,NaN,40.9,...,NaN,MRPMT,MRPMT-2022.html,2022,NaN,24.59,29.81,NaN,NaN,NaN
8335,"Jan 1, 1:00 am",-4.0,NaN,NaN,NaN,-22.0,SW,12G19,NaN,40.7,...,NaN,MRPMT,MRPMT-2022.html,2022,NaN,24.57,29.79,NaN,NaN,NaN


In [20]:
# rename columns, watch out for misordering due to file names in unordered set!
new_cols = {
    all_wx.columns[0]: "dt",
    all_wx.columns[1]: "temp",
    'DewPoint(°F)': "dew_pt",
    'RelativeHumidity(%)': "rH",
    'HeatIndex(°F)': "heat_idx",
    'WindChill(°F)': "wind_chill",
    'WindDirection': "wind_dir",
    'WindSpeed(mph)' : "wind_spd",
    'SnowDepth(in)': "snow_depth",
    'Snowfall3 hour(in)': "snowfall_3hr",
    'Snowfall6 Hour(in)': "snowfall_6hr",
    'Snowfall24 Hour(in)': "snowfall_24hr",
    'Sea LevelPressure(mb)': "sea_lvl_press",
    'StationPressure(in Hg)': "sta_press",
    'AltimeterSetting(in Hg)': "altimeter_setting",
    'SolarRadiation(W/m²)': "solar_radiation",
    'PercentPossible(%)': "pct_possible",
    '1 HourPrecip(in)': "hr_precip"
}

all_wx.rename(columns=new_cols, inplace = True)
all_wx.drop(columns='Unnamed: 1', inplace=True)
all_wx.sample(4)

,dt,temp,dew_pt,rH,heat_idx,wind_chill,wind_dir,wind_spd,hr_precip,snow_depth,...,snowfall_6hr,snowfall_24hr,site,file_name,year,sea_lvl_press,sta_press,altimeter_setting,solar_radiation,pct_possible
5804,"Apr 17, 9:00 am",37.0,NaN,NaN,NaN,28.0,SW,14G28,NaN,75.8,...,0.0,0.0,MRPMT,MRPMT-2020.html,2020,NaN,24.66,29.9,NaN,NaN
1580,"Oct 27, 7:00 pm",17.0,15.0,91.0,NaN,8.0,SW,6G11,NaN,0.0,...,0.0,0.0,SH7MT,SH7MT-2019.html,2019,NaN,NaN,NaN,1.0,4 %
5193,"May 27, 10:00 am",41.0,38.0,89.0,NaN,NaN,WSW,3G8,NaN,17.7,...,0.0,0.0,S11MT,S11MT-2020.html,2020,NaN,NaN,NaN,NaN,NaN
887,"Nov 25, 5:00 pm",28.0,23.0,80.0,NaN,16.0,SSW,15G30,NaN,13.1,...,0.0,0.0,SH7MT,SH7MT-2021.html,2021,NaN,NaN,NaN,9.0,10 %


In [21]:
all_wx.dt = [f'{d}, {y}' for d, y in zip(all_wx.dt, all_wx.year)]
all_wx.dt = pd.to_datetime(all_wx.dt, format='%b %d, %I:%M %p, %Y', errors='coerce') # parse the datetime
all_wx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203973 entries, 17 to 8336
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   dt                 203973 non-null  datetime64[ns]
 1   temp               203973 non-null  float64       
 2   dew_pt             172680 non-null  float64       
 3   rH                 172612 non-null  float64       
 4   heat_idx           3540 non-null    float64       
 5   wind_chill         71720 non-null   float64       
 6   wind_dir           193006 non-null  object        
 7   wind_spd           203973 non-null  object        
 8   hr_precip          51070 non-null   float64       
 9   snow_depth         128475 non-null  float64       
 10  snowfall_3hr       127208 non-null  float64       
 11  snowfall_6hr       126826 non-null  float64       
 12  snowfall_24hr      126391 non-null  float64       
 13  site               203973 non-null  object   

In [22]:
all_wx.reset_index(drop=True, inplace=True)
all_wx

,dt,temp,dew_pt,rH,heat_idx,wind_chill,wind_dir,wind_spd,hr_precip,snow_depth,...,snowfall_6hr,snowfall_24hr,site,file_name,year,sea_lvl_press,sta_press,altimeter_setting,solar_radiation,pct_possible
0,2022-12-23 04:00:00,-25.0,-31.0,72.0,NaN,NaN,ESE,1G4,0.0,12.6,...,0.1,0.3,JVEMT,JVEMT-2022.html,2022,NaN,NaN,NaN,NaN,NaN
1,2022-12-23 03:00:00,-26.0,-32.0,72.0,NaN,NaN,ESE,1G3,0.0,12.6,...,0.1,0.4,JVEMT,JVEMT-2022.html,2022,NaN,NaN,NaN,NaN,NaN
2,2022-12-23 02:00:00,-27.0,-33.0,71.0,NaN,NaN,ESE,1G4,0.0,12.5,...,0.0,0.3,JVEMT,JVEMT-2022.html,2022,NaN,NaN,NaN,NaN,NaN
3,2022-12-23 01:00:00,-28.0,-34.0,72.0,NaN,NaN,ESE,2G5,0.0,12.5,...,0.0,0.2,JVEMT,JVEMT-2022.html,2022,NaN,NaN,NaN,NaN,NaN
4,2022-12-23 00:00:00,-29.0,-35.0,71.0,NaN,NaN,ESE,2G4,0.0,12.5,...,0.0,0.2,JVEMT,JVEMT-2022.html,2022,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203968,2022-01-01 04:00:00,-4.0,NaN,NaN,NaN,-26.0,S,18G31,NaN,40.8,...,0.0,NaN,MRPMT,MRPMT-2022.html,2022,NaN,24.60,29.82,NaN,NaN
203969,2022-01-01 03:00:00,-5.0,NaN,NaN,NaN,-26.0,S,15G21,NaN,41.5,...,0.3,NaN,MRPMT,MRPMT-2022.html,2022,NaN,24.59,29.81,NaN,NaN
203970,2022-01-01 02:00:00,-3.0,NaN,NaN,NaN,-23.0,SSW,15G25,NaN,40.9,...,0.4,NaN,MRPMT,MRPMT-2022.html,2022,NaN,24.59,29.81,NaN,NaN
203971,2022-01-01 01:00:00,-4.0,NaN,NaN,NaN,-22.0,SW,12G19,NaN,40.7,...,1.0,NaN,MRPMT,MRPMT-2022.html,2022,NaN,24.57,29.79,NaN,NaN


In [ ]:
import pandas_profiling

# profile = all_wx.profile_report()
# profile.to_file('all_wx-profile.html')

In [ ]:
all_wx.to_pickle("pkl/all_wx.pkl")  